In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from metricas import *
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

def read_predictions(file, col):
    df = pd.read_csv(file, index_col = 0)
    df.index = pd.to_datetime(df.index)
    df = df[["Pred"]]
    df.columns = [col]
    return df

persistencia = pd.read_csv('Resultados/resultados_baseline.csv', index_col = 0)
persistencia.index = pd.to_datetime(persistencia.index)
persistencia = persistencia[["Real", "Pred"]]

rf = read_predictions('Resultados/resultados_rf.csv', 'RF')
xgboost = read_predictions('Resultados/resultados_xgboost.csv', 'XGBoost')
xgboost_stacked = read_predictions('Resultados/resultados_xgboost_stack.csv', 'XGBoost')
xgboost = pd.concat([xgboost_stacked.loc["2020"], xgboost])
reg_lin = read_predictions('Resultados/resultados_regresion_lineal.csv', 'Reg_Lin')
lasso = read_predictions('Resultados/resultados_reg_lasso.csv', 'Lasso')
ridge = read_predictions('Resultados/resultados_reg_ridge.csv', 'Ridge')


In [2]:
stacked = lasso.reset_index().merge(ridge.reset_index())\
        .merge(reg_lin.reset_index())\
        .merge(persistencia.reset_index(), left_on = 'index', right_on = 'Fecha')\
        .merge(rf.reset_index())\
        .merge(xgboost.reset_index())

stacked = stacked.drop(columns = 'index')
stacked = stacked.set_index('Fecha')

# Regresión Lineal

In [3]:
import statsmodels.api as sm

df_stacked = stacked[["Reg_Lin", "XGBoost", "Lasso", "Ridge", "Real", "RF"]]

In [6]:
def bascktesting_regresion_lineal(db, año, mes, rolling_window, days_start):
    fecha = datetime(año, mes, 1)
    fecha_str = str(año) + '-' + str(mes) + '-' + str(1)
    
    db = db.dropna()
    array_real, array_pred, array_date = np.zeros(len(db.loc[fecha_str:])), np.zeros(len(db.loc[fecha_str:])), []

    # En caso de que haya rolling window
    if rolling_window == True:
        for day in range(int(len(db.loc[fecha_str:]) / 24)):
            fecha_end_train = fecha + relativedelta(days = day)
            fecha_start_train = fecha_end_train - relativedelta(days = days_start)
            fecha_end_train = str(fecha_end_train.year) + "-" + str(fecha_end_train.month) + "-" + str(fecha_end_train.day)
            fecha_start_train = str(fecha_start_train.year) + "-" + str(fecha_start_train.month) + "-" + str(fecha_start_train.day)
            db_aux = db.loc[fecha_start_train:fecha_end_train]
            
            # Dataframe de Entrenamiento
            X_train = sm.add_constant(db_aux)
            y_train = X_train.pop('Real')
            
            # Dataframe de Test
            X_test = X_train.loc[fecha_end_train]
            y_test = y_train.pop(fecha_end_train)

            # Quitamos al dataframe de train, el dataframe de test
            X_train = X_train.drop(index = X_train.iloc[-24:].index)

            # Entrenamos el modelo
            model = sm.OLS(y_train, X_train).fit()
            
            # Realizamos las predicciones
            pred = model.predict(X_test)
            date = pred.index

            array_date.append(date)
            array_pred[day*24:day*24 + 24] = pred
            array_real[day*24:day*24 + 24] = y_test
        
    # En caso de que no haya rolling window, siempre cogemos todo el dataframe disponible
    else:
        for day in range(int(len(db.loc[fecha_str:]) / 24)):
            fecha_end_train = fecha + relativedelta(days = day)
            fecha_end_train = str(fecha_end_train.year) + "-" + str(fecha_end_train.month) + "-" + str(fecha_end_train.day)
            db_aux = db.loc[:fecha_end_train]

            # Dataframe de Entrenamiento
            X_train = sm.add_constant(db_aux)
            y_train = X_train.pop('Real')
            
            # Dataframe de Test
            X_test = X_train.loc[fecha_end_train]
            y_test = y_train.pop(fecha_end_train)
            
            # Quitamos al dataframe de train, el dataframe de test
            X_train = X_train.drop(index = X_train.iloc[-24:].index)
            
            # Entrenamos el modelo
            model = sm.OLS(y_train, X_train).fit()
            
            # Realizamos las predicciones
            pred = model.predict(X_test)
            date = pred.index

            array_date.append(date)
            array_pred[day*24:day*24 + 24] = pred
            array_real[day*24:day*24 + 24] = y_test
            
    array_date = [item for sublist in array_date for item in sublist]
    return array_pred, array_real, array_date
    
pred, real, dates = bascktesting_regresion_lineal(df_stacked, 2021, 1, rolling_window = False, days_start = 150)

In [121]:
# Backtesting rapido

for dias in [7, 14, 30, 60, 90, 150, 220, 365]:
    print(dias)
    pred, real, dates = bascktesting_regresion_lineal(df_stacked, 2021, 1, rolling_window = True, days_start = dias)
    resultados = pd.DataFrame({"Pred":pred, "Real":real}, index = dates)
    resultados["Pred"][resultados.loc[:, "Pred"] < 0] = 0
    calculo_metricas(resultados)

7
     MAE  MAE (median)  MAPE  WMAPE   RMSE  % Trend
0  10.48          6.46  38.0   9.36  17.36    82.34
14
     MAE  MAE (median)   MAPE  WMAPE   RMSE  % Trend
0  10.03          6.16  41.82   8.96  16.51    82.98
30
    MAE  MAE (median)   MAPE  WMAPE   RMSE  % Trend
0  9.33          5.88  86.69   8.34  15.11    84.61
60
    MAE  MAE (median)    MAPE  WMAPE   RMSE  % Trend
0  9.08          5.66  100.03   8.11  15.03    84.59
90
    MAE  MAE (median)   MAPE  WMAPE   RMSE  % Trend
0  9.04          5.66  83.29   8.07  14.82    84.86
150
    MAE  MAE (median)   MAPE  WMAPE   RMSE  % Trend
0  9.04          5.68  81.48   8.08  14.82    84.85
220
    MAE  MAE (median)   MAPE  WMAPE   RMSE  % Trend
0  9.06          5.67  80.58    8.1  14.87    84.87
365
    MAE  MAE (median)   MAPE  WMAPE   RMSE  % Trend
0  9.04          5.62  81.58   8.08  14.81    84.99


In [7]:
resultados = pd.DataFrame({"Pred":pred, "Real":real}, index = dates)
resultados["Pred"][resultados.loc[:, "Pred"] < 0] = 0


reg_lineal, reg_lineal_metricas_mensuales, met = calculo_metricas(resultados)

    MAE  MAE (median)   MAPE  WMAPE  RMSE  % Trend
0  9.04          5.63  81.58   8.07  14.8     85.0


In [124]:
reg_lineal.to_csv('Resultados/stacked_model_reg_lin.csv')

# XGBoost

In [5]:
import xgboost as xgb

def backtesting_xgboost(db, año, mes, rolling_window, days_start, target_mean):
    fecha = datetime(año, mes, 1)
    fecha_str = str(año) + '-' + str(mes) + '-' + str(1)
    
    db = db.dropna()
    array_real, array_pred, array_date = np.zeros(len(db.loc[fecha_str:])), np.zeros(len(db.loc[fecha_str:])), []

    # En caso de que haya rolling window
    if rolling_window == True:
        for day in range(int(len(db.loc[fecha_str:]) / 24)):
            print(day)
            fecha_end_train = fecha + relativedelta(days = day)
            fecha_start_train = fecha_end_train - relativedelta(days = days_start)
            fecha_end_train = str(fecha_end_train.year) + "-" + str(fecha_end_train.month) + "-" + str(fecha_end_train.day)
            fecha_start_train = str(fecha_start_train.year) + "-" + str(fecha_start_train.month) + "-" + str(fecha_start_train.day)
            db_aux = db.loc[fecha_start_train:fecha_end_train]
            
            # Dataframe de Entrenamiento
            X_train = db_aux
            
            
            # Dataframe de Test
            X_test = X_train.loc[fecha_end_train]
            

            # Quitamos al dataframe de train, el dataframe de test
            X_train = X_train.drop(index = X_test.index)
            
            # Target mean
            if target_mean:
                X_train = X_train.drop(columns = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes',
                                               'Sabado', 'Domingo', 'Hora_1', 'Hora_2', 'Hora_3', 'Hora_4', 'Hora_5',
                                               'Hora_6', 'Hora_7', 'Hora_8', 'Hora_9', 'Hora_10', 'Hora_11', 'Hora_12',
                                               'Hora_13', 'Hora_14', 'Hora_15', 'Hora_16', 'Hora_17', 'Hora_18',
                                               'Hora_19', 'Hora_20', 'Hora_21', 'Hora_22', 'Hora_23', 'Hora_24'])
                
                X_test = X_test.drop(columns = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes',
                                               'Sabado', 'Domingo', 'Hora_1', 'Hora_2', 'Hora_3', 'Hora_4', 'Hora_5',
                                               'Hora_6', 'Hora_7', 'Hora_8', 'Hora_9', 'Hora_10', 'Hora_11', 'Hora_12',
                                               'Hora_13', 'Hora_14', 'Hora_15', 'Hora_16', 'Hora_17', 'Hora_18',
                                               'Hora_19', 'Hora_20', 'Hora_21', 'Hora_22', 'Hora_23', 'Hora_24'])
                dic_hora , dic_dia = {} , {}
                for i in range(24):
                    dic_hora[X_train.groupby([X_train.index.hour])[["Real"]].mean().reset_index().iloc[i,0]] = \
                             X_train.groupby([X_train.index.hour])[["Real"]].mean().reset_index().iloc[i,1]
                    
                # Hour encoded para train y test
                X_train.loc[:, "Hour_encoded"] = [dic_hora[dato] for dato in X_train.index.hour]
                X_test.loc[:, "Hour_encoded"] = [dic_hora[dato] for dato in X_test.index.hour]
                             
                for i in range(7):
                    dic_dia[X_train.groupby([X_train.index.weekday])[["Real"]].mean().reset_index().iloc[i,0]] = \
                            X_train.groupby([X_train.index.weekday])[["Real"]].mean().reset_index().iloc[i,1]
                    
                X_train.loc[:, "Day_encoded"] = [dic_dia[dato] for dato in X_train.index.weekday]
                X_test.loc[:, "Day_encoded"] = [dic_dia[dato] for dato in X_test.index.weekday]
                    
            # Quitamos la variable objetivo
            y_train = X_train.pop('Real')
            y_test = X_test.pop('Real')

            # Entrenamos el modelo
            model = xgb.XGBRegressor(n_estimators=1000,
                                     max_depth=None,
                                     eta=0.03,
                                     subsample=0.7,
                                     colsample_bytree=0.8).fit(X_train,y_train)
            
            # Realizamos las predicciones
            pred = model.predict(X_test)
            date = X_test.index

            array_date.append(date)
            array_pred[day*24:day*24 + 24] = pred
            array_real[day*24:day*24 + 24] = y_test
        
    # En caso de que no haya rolling window, siempre cogemos todo el dataframe disponible
    else:
        for day in range(int(len(db.loc[fecha_str:]) / 24)):
            print(day)
            fecha_end_train = fecha + relativedelta(days = day)
            fecha_end_train = str(fecha_end_train.year) + "-" + str(fecha_end_train.month) + "-" + str(fecha_end_train.day)
            db_aux = db.loc[:fecha_end_train]

            # Dataframe de Entrenamiento
            X_train = db_aux
            y_train = X_train.pop('Real')
            
            # Dataframe de Test
            X_test = X_train.loc[fecha_end_train]
            y_test = y_train.pop(fecha_end_train)
            
            # Quitamos al dataframe de train, el dataframe de test
            X_train = X_train.drop(index = X_train.iloc[-24:].index)
            
            # Entrenamos el modelo
            model = xgb.XGBRegressor(n_estimators=1000,
                                     max_depth=None,
                                     eta=0.03,
                                     subsample=0.7,
                                     colsample_bytree=0.8).fit(X_train,y_train)
            
            # Realizamos las predicciones
            pred = model.predict(X_test)
            date = X_test.index

            array_date.append(date)
            array_pred[day*24:day*24 + 24] = pred
            array_real[day*24:day*24 + 24] = y_test
            
    array_date = [item for sublist in array_date for item in sublist]
    return array_pred, array_real, array_date

pred, real, dates = backtesting_xgboost(df_stacked, 2021, 1,
                                        rolling_window = False, days_start = 30, target_mean = False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [6]:
resultados = pd.DataFrame({"Pred":pred, "Real":real}, index = dates)
resultados["Pred"][resultados.loc[:, "Pred"] < 0] = 0


xgboost, xgboost_metricas_mensuales, met = calculo_metricas(resultados)

     MAE  MAE (median)   MAPE  WMAPE   RMSE  % Trend
0  10.08          6.25  86.46   9.01  16.22    74.29


# Random Forest

In [7]:
from sklearn.ensemble import RandomForestRegressor

def bascktesting_random_forest(db, año, mes, rolling_window, days_start, target_mean = False):
    fecha = datetime(año, mes, 1)
    fecha_str = str(año) + '-' + str(mes) + '-' + str(1)
    
    db = db.dropna()
    array_real, array_pred, array_date = np.zeros(len(db.loc[fecha_str:])), np.zeros(len(db.loc[fecha_str:])), []

    # En caso de que haya rolling window
    if rolling_window == True:
        for day in range(int(len(db.loc[fecha_str:]) / 24)):
            print(day)
            fecha_end_train = fecha + relativedelta(days = day)
            fecha_start_train = fecha_end_train - relativedelta(days = days_start)
            fecha_end_train = str(fecha_end_train.year) + "-" + str(fecha_end_train.month) + "-" + str(fecha_end_train.day)
            fecha_start_train = str(fecha_start_train.year) + "-" + str(fecha_start_train.month) + "-" + str(fecha_start_train.day)
            db_aux = db.loc[fecha_start_train:fecha_end_train]
            
            # Dataframe de Entrenamiento
            X_train = db_aux
            
            
            # Dataframe de Test
            X_test = X_train.loc[fecha_end_train]
            

            # Quitamos al dataframe de train, el dataframe de test
            X_train = X_train.drop(index = X_test.index)
            
            # Target mean
            if target_mean:
                X_train = X_train.drop(columns = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes',
                                               'Sabado', 'Domingo', 'Hora_1', 'Hora_2', 'Hora_3', 'Hora_4', 'Hora_5',
                                               'Hora_6', 'Hora_7', 'Hora_8', 'Hora_9', 'Hora_10', 'Hora_11', 'Hora_12',
                                               'Hora_13', 'Hora_14', 'Hora_15', 'Hora_16', 'Hora_17', 'Hora_18',
                                               'Hora_19', 'Hora_20', 'Hora_21', 'Hora_22', 'Hora_23', 'Hora_24'])
                
                X_test = X_test.drop(columns = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes',
                                               'Sabado', 'Domingo', 'Hora_1', 'Hora_2', 'Hora_3', 'Hora_4', 'Hora_5',
                                               'Hora_6', 'Hora_7', 'Hora_8', 'Hora_9', 'Hora_10', 'Hora_11', 'Hora_12',
                                               'Hora_13', 'Hora_14', 'Hora_15', 'Hora_16', 'Hora_17', 'Hora_18',
                                               'Hora_19', 'Hora_20', 'Hora_21', 'Hora_22', 'Hora_23', 'Hora_24'])
                dic_hora , dic_dia = {} , {}
                for i in range(24):
                    dic_hora[X_train.groupby([X_train.index.hour])[["Real"]].mean().reset_index().iloc[i,0]] = \
                             X_train.groupby([X_train.index.hour])[["Real"]].mean().reset_index().iloc[i,1]
                    
                # Hour encoded para train y test
                X_train.loc[:, "Hour_encoded"] = [dic_hora[dato] for dato in X_train.index.hour]
                X_test.loc[:, "Hour_encoded"] = [dic_hora[dato] for dato in X_test.index.hour]
                             
                for i in range(7):
                    dic_dia[X_train.groupby([X_train.index.weekday])[["Real"]].mean().reset_index().iloc[i,0]] = \
                            X_train.groupby([X_train.index.weekday])[["Real"]].mean().reset_index().iloc[i,1]
                    
                X_train.loc[:, "Day_encoded"] = [dic_dia[dato] for dato in X_train.index.weekday]
                X_test.loc[:, "Day_encoded"] = [dic_dia[dato] for dato in X_test.index.weekday]
                    
            # Quitamos la variable objetivo
            y_train = X_train.pop('Real')
            y_test = X_test.pop('Real')

            # Entrenamos el modelo
            model = RandomForestRegressor(n_estimators = 150,
                              criterion = "mae",
                              max_depth = None,
                              max_features = 2,
                              n_jobs = -1,
                              warm_start = False,
                              random_state = 123).fit(X_train,y_train)
            
            # Realizamos las predicciones
            pred = model.predict(X_test)
            date = X_test.index

            array_date.append(date)
            array_pred[day*24:day*24 + 24] = pred
            array_real[day*24:day*24 + 24] = y_test
        
    # En caso de que no haya rolling window, siempre cogemos todo el dataframe disponible
    else:
        for day in range(int(len(db.loc[fecha_str:]) / 24)):
            print(day)
            fecha_end_train = fecha + relativedelta(days = day)
            fecha_end_train = str(fecha_end_train.year) + "-" + str(fecha_end_train.month) + "-" + str(fecha_end_train.day)
            db_aux = db.loc[:fecha_end_train]

            # Dataframe de Entrenamiento
            X_train = db_aux
            y_train = X_train.pop('Real')
            
            # Dataframe de Test
            X_test = X_train.loc[fecha_end_train]
            y_test = y_train.pop(fecha_end_train)
            
            # Quitamos al dataframe de train, el dataframe de test
            X_train = X_train.drop(index = X_train.iloc[-24:].index)
            
            # Entrenamos el modelo
            model = RandomForestRegressor(n_estimators = 100,
                              criterion = "mae",
                              max_depth = None,
                              max_features = X_train.shape[1] - 1,
#                               n_jobs = -1,
                              random_state = 123).fit(X_train,y_train)
            
            # Realizamos las predicciones
            pred = model.predict(X_test)
            date = X_test.index

            array_date.append(date)
            array_pred[day*24:day*24 + 24] = pred
            array_real[day*24:day*24 + 24] = y_test
            
    array_date = [item for sublist in array_date for item in sublist]
    return array_pred, array_real, array_date

pred, real, dates = bascktesting_random_forest(df_stacked, 2021, 1,
                                               rolling_window = False, days_start = 30, target_mean = False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [8]:
resultados = pd.DataFrame({"Pred":pred, "Real":real}, index = dates)
resultados["Pred"][resultados.loc[:, "Pred"] < 0] = 0


rf, rf_metricas_mensuales, met = calculo_metricas(resultados)

    MAE  MAE (median)   MAPE  WMAPE   RMSE  % Trend
0  9.99          6.22  90.15   8.93  16.02    75.18


# Ridge

In [4]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV

def bascktesting_ridge(db, año, mes, rolling_window, days_start):
    fecha = datetime(año, mes, 1)
    fecha_str = str(año) + '-' + str(mes) + '-' + str(1)
    
    db = db.dropna()
    array_real, array_pred, array_date = np.zeros(len(db.loc[fecha_str:])), np.zeros(len(db.loc[fecha_str:])), []

    # En caso de que haya rolling window
    if rolling_window == True:
        for day in range(int(len(db.loc[fecha_str:]) / 24)):
            fecha_end_train = fecha + relativedelta(days = day)
            fecha_start_train = fecha_end_train - relativedelta(days = days_start)
            fecha_end_train = str(fecha_end_train.year) + "-" + str(fecha_end_train.month) + "-" + str(fecha_end_train.day)
            fecha_start_train = str(fecha_start_train.year) + "-" + str(fecha_start_train.month) + "-" + str(fecha_start_train.day)
            db_aux = db.loc[fecha_start_train:fecha_end_train]
            
            # Dataframe de Entrenamiento
            X_train = db_aux.copy()
            y_train = X_train.pop('Real')
            
            # Dataframe de Test
            X_test = X_train.loc[fecha_end_train]
            y_test = y_train.pop(fecha_end_train)

            # Quitamos al dataframe de train, el dataframe de test
            X_train = X_train.drop(index = X_train.iloc[-24:].index)

            # Entrenamos el model
            model = RidgeCV(
            alphas          = np.logspace(-10, 5, 200),
            fit_intercept   = True,
            normalize       = True,
            store_cv_values = True
             )

            _ = model.fit(X = X_train, y = y_train)
            
            # Realizamos las predicciones
            pred = model.predict(X_test)
            date = X_test.index

            array_date.append(date)
            array_pred[day*24:day*24 + 24] = pred
            array_real[day*24:day*24 + 24] = y_test
    # En caso de que no haya rolling window, siempre cogemos todo el dataframe disponible
    else:
        for day in range(int(len(db.loc[fecha_str:]) / 24)):
            fecha_end_train = fecha + relativedelta(days = day)
            fecha_end_train = str(fecha_end_train.year) + "-" + str(fecha_end_train.month) + "-" + str(fecha_end_train.day)
            db_aux = db.loc[:fecha_end_train]

            # Dataframe de Entrenamiento
            X_train = db_aux.copy()
            y_train = X_train.pop('Real')
            
            # Dataframe de Test
            X_test = X_train.loc[fecha_end_train]
            y_test = y_train.pop(fecha_end_train)
            
            # Quitamos al dataframe de train, el dataframe de test
            X_train = X_train.drop(index = X_train.iloc[-24:].index)
            
            # Entrenamos el model
            model = RidgeCV(
            alphas          = np.logspace(-10, 5, 200),
            fit_intercept   = True,
            normalize       = True,
            store_cv_values = True
             )

            _ = model.fit(X = X_train, y = y_train)
            
            # Realizamos las predicciones
            pred = model.predict(X_test)
            date = X_test.index

            array_date.append(date)
            array_pred[day*24:day*24 + 24] = pred
            array_real[day*24:day*24 + 24] = y_test

    array_date = [item for sublist in array_date for item in sublist]
    return array_pred, array_real, array_date
    
pred, real, dates = bascktesting_ridge(df_stacked, 2021, 1, rolling_window = False, days_start = 120)

In [126]:
for dias in [7, 14, 21, 30, 45, 60, 90 ,120, 150, 220, 365]:
    print(dias)
    print("**********************************")
    pred, real, dates = bascktesting_ridge(df_stacked, 2021, 1, rolling_window = True, days_start = dias)
    resultados = pd.DataFrame({"Pred":pred, "Real":real}, index = dates)
    resultados["Pred"][resultados.loc[:, "Pred"] < 0] = 0
    calculo_metricas(resultados)

7
**********************************
    MAE  MAE (median)   MAPE  WMAPE   RMSE  % Trend
0  10.4          6.45  38.25   9.29  17.22    82.52
14
**********************************
     MAE  MAE (median)  MAPE  WMAPE   RMSE  % Trend
0  10.03          6.13  42.1   8.96  16.56    83.03
21
**********************************
    MAE  MAE (median)  MAPE  WMAPE   RMSE  % Trend
0  9.69          5.97  51.6   8.66  15.98    83.97
30
**********************************
    MAE  MAE (median)   MAPE  WMAPE   RMSE  % Trend
0  9.33          5.88  84.78   8.33  15.13    84.64
45
**********************************
    MAE  MAE (median)    MAPE  WMAPE   RMSE  % Trend
0  9.12           5.8  100.21   8.14  14.91    84.32
60
**********************************
    MAE  MAE (median)    MAPE  WMAPE   RMSE  % Trend
0  9.07          5.63  100.52    8.1  15.03     84.6
90
**********************************
    MAE  MAE (median)   MAPE  WMAPE   RMSE  % Trend
0  9.04          5.66  83.39   8.07  14.82    84.88
120
*

In [5]:
resultados = pd.DataFrame({"Pred":pred, "Real":real}, index = dates)
resultados["Pred"][resultados.loc[:, "Pred"] < 0] = 0


ridge, ridge_metricas_mensuales, met = calculo_metricas(resultados)

    MAE  MAE (median)   MAPE  WMAPE   RMSE  % Trend
0  9.04          5.64  81.69   8.08  14.81    85.03


In [129]:
ridge.to_csv('Resultados/stacked_model_ridge.csv')